In [1]:
import os

In [13]:
lang = "annotation_bokmaal"
finished = "FINISHED_ANNOTATIONS"
path = os.path.join(os.getcwd(), lang, finished)

In [14]:
from collections import defaultdict
files_with_dir = defaultdict(list)
# walk through the directory and get all the files
# add their paths to the dictionary

for root, dirs, files in os.walk(path):
    for file in files:
        if ".ann" in file: # disregard txts, assuming 1 to 1
            # only append the path from the "FINISHED_ANNOTATION" folder:
            short_path = root.split(finished)[1]
            files_with_dir[file].append(os.path.join(root, file))

In [15]:
import json
# save the dictionary to a json file
with open("finished_bm.json", "w") as f:
    json.dump(files_with_dir, f, indent=4)

# Process the data

In [16]:
s = "bt~BT-20121106-2792372_review_liljacs.ann"
def strip_annotator(fileid):
    fileid, filetype = fileid.split(".")
    isnum = False
    while not isnum:
        if fileid[-1].isdigit():
            isnum = True
        else:
            fileid = fileid[:-1]
    return fileid + "." + filetype
strip_annotator(s)

'bt~BT-20121106-2792372.ann'

In [17]:
def bokmaal_sort(path):
    # prioritization:
    # path contains...
    # 1. "Review_and_Curation"
    # 2. "IAA_"
    if "Review_and_Curation" in path:
        return 1
    elif "IAA_" in path:
        return 2
    else:
        return 3

first_filter = {}
for fileid, paths in files_with_dir.items():
    if "_" in fileid:
        fileid = strip_annotator(fileid)

    if len(paths) > 1:
        sorted_paths = sorted(paths, key=bokmaal_sort)

        first_filter[fileid] = sorted_paths[0]
    else:
        first_filter[fileid] = paths[0]

with open("finished_bm.json", "w") as f:
    json.dump(first_filter, f, indent=4)

In [19]:
save_bm = "FINISHED_BOKMAAL"
save_nn = "FINISHED_NYNORSK"
os.makedirs(save_bm, exist_ok=True)
os.makedirs(save_nn, exist_ok=True)

In [20]:
import shutil
for fileid, ann_path in first_filter.items():
    txt_path = ann_path.replace(".ann", ".txt")
    txt_id = os.path.basename(txt_path)
    # copy the files to the new folder
    shutil.copy(ann_path, os.path.join(save_bm, fileid))
    shutil.copy(txt_path, os.path.join(save_bm, txt_id))

